In [45]:
import pandas as pd
import matplotlib as plt
import re
import numpy as np

In [ ]:
df = pd.read_csv("./mercadoimobiliario.csv", sep= ';')
# extração e correção variavel valor
df['Valor'] = df['Valor'].str.replace(',', ".")
df['Valor'].str.strip()
df['Valor'] = pd.to_numeric(df['Valor'])

df['Data'] = pd.to_datetime(df['Data'], dayfirst=True, format='%d/%m/%Y')
print(df.columns)

df.columns = df.columns.str.strip()

df['tipo_transacao'] = df['Info'].apply(lambda x: x.split('_')[0])
df['estado'] = df['Info'].apply(lambda x: x.split('_')[-1])
df['Info'] = df['Info'].apply(lambda x: '_'.join(x.split('_')[:-1]))

df_organizado_data = df.sort_values(by='Data')
df_organizado_data.columns = ['data', 'info', 'valor', 'tipo_transacao','estado', 'tipo_credito']
df_organizado_data = df_organizado_data.reset_index(drop=True)

df['tipo_transacao'].unique()

In [3]:
# Trabalhando em cada operação do Dataframe, começando por imóveis
filtro_imoveis = df_organizado_data[df_organizado_data['info'].str.startswith('imoveis')]

In [22]:
filtro_imoveis['info'].nunique()

392

In [23]:
filtro_imoveis.nunique()

data       76
info      392
valor    8264
dtype: int64

In [24]:
#criando uma função para encontrar as unidades federativas e alocar em um DataFrame
def filtro_unidade_federativa(dataframe,unidade_federativa):
    return dataframe[dataframe['info'].str.endswith(unidade_federativa)]

In [25]:
estados_br = [
    "ac", "al", "ap", "am", "ba", "ce", "df", "es",
    "go", "ma", "mt", "ms", "mg", "pa", "pb", "pr",
    "pe", "pi", "rj", "rn", "rs", "ro", "rr", "sc",
    "sp", "se", "to"
]

In [26]:

dicionario_estados = {}

In [27]:
for estados in estados_br:
    dicionario_estados[estados] = filtro_unidade_federativa(filtro_imoveis, estados)

In [28]:
dicionario_estados

{'ac':              data                                      info      valor
 113893 2018-01-31                 imoveis_area_privativa_ac     121.52
 114110 2018-01-31                      imoveis_tipo_casa_ac       4.00
 114236 2018-01-31               imoveis_tipo_apartamento_ac      10.00
 114511 2018-01-31                     imoveis_area_total_ac     125.76
 114846 2018-01-31                imoveis_valor_avaliacao_ac  216000.00
 ...           ...                                       ...        ...
 348722 2024-04-30              imoveis_dormitorio_4_mais_ac       3.00
 348775 2024-04-30  imoveis_garantia_alienacao_fiduciaria_ac      31.00
 349090 2024-04-30                imoveis_valor_avaliacao_ac  305000.00
 349209 2024-04-30                   imoveis_dormitorio_1_ac       2.00
 349260 2024-04-30            imoveis_implantacao_isolado_ac      21.00
 
 [1026 rows x 3 columns],
 'al':              data                               info      valor
 113921 2018-01-31          imo

In [23]:
#dicionario_estados['sp'].loc[:,'info'] = dicionario_estados['sp']['info'].str.strip()
#filtro_dormitorios_3 = dicionario_estados['sp'][~dicionario_estados['sp']['info'].str.contains(r'\wmais\w|\wgarantia\w', regex = True, case=False)]
#filtro_dormitorios_3 = filtro_dormitorios_3[filtro_dormitorios_3['info'].str.contains(r'\wdormitorio\w', regex = True, case=False)]
#filtro_imoveis_sp['Info'].str.split('_', expand=True)

In [29]:
filtro_dormitorios = {}

In [30]:
for estado in estados_br:
    dicionario_estados[estado].loc[:,'info'] = dicionario_estados[estado]['info'].str.strip()
    filtro_dormitorios[estado] = dicionario_estados[estado][~dicionario_estados[estado]['info'].str.contains(r'\wmais\w|\wgarantia\w', regex = True, case=False)]
    filtro_dormitorios[estado]= filtro_dormitorios[estado][filtro_dormitorios[estado]['info'].str.contains(r'\wdormitorio\w', regex = True, case=False)]
    filtro_dormitorios[estado][['identificador','descritivo', 'quantidade', 'uf']] = filtro_dormitorios[estado]['info'].str.split('_', expand = True)


In [31]:
filtro_dormitorios['sp']

,data,info,valor,identificador,descritivo,quantidade,uf
115297,2018-01-31,imoveis_dormitorio_1_sp,3630.0,imoveis,dormitorio,1,sp
115472,2018-01-31,imoveis_dormitorio_2_sp,9977.0,imoveis,dormitorio,2,sp
116387,2018-01-31,imoveis_dormitorio_3_sp,2840.0,imoveis,dormitorio,3,sp
118097,2018-02-28,imoveis_dormitorio_1_sp,3534.0,imoveis,dormitorio,1,sp
118372,2018-02-28,imoveis_dormitorio_3_sp,2486.0,imoveis,dormitorio,3,sp
...,...,...,...,...,...,...,...
344903,2024-03-31,imoveis_dormitorio_1_sp,2577.0,imoveis,dormitorio,1,sp
346294,2024-03-31,imoveis_dormitorio_3_sp,3445.0,imoveis,dormitorio,3,sp
347647,2024-04-30,imoveis_dormitorio_1_sp,1894.0,imoveis,dormitorio,1,sp
348057,2024-04-30,imoveis_dormitorio_3_sp,3533.0,imoveis,dormitorio,3,sp


In [51]:
#utilizando Regular Expressions para filtrar os valores problemáticos
#filtro_imoveis_4_mais = filtro_imoveis_sp[~filtro_imoveis_sp['Info'].str.contains(r'\wmais\w|\wgarantia\w', regex=True, case= False)]
#filtro_imoveis_4_mais = filtro_imoveis_4_mais[filtro_imoveis_4_mais['Info'].str.contains(r'\wdormitorio\w', regex=True, case=False)]




In [54]:
filtro_dormitorios_3

,data,info,valor
115297,2018-01-31,imoveis_dormitorio_1_sp,3630.0
115472,2018-01-31,imoveis_dormitorio_2_sp,9977.0
116387,2018-01-31,imoveis_dormitorio_3_sp,2840.0
118097,2018-02-28,imoveis_dormitorio_1_sp,3534.0
118372,2018-02-28,imoveis_dormitorio_3_sp,2486.0
...,...,...,...
344903,2024-03-31,imoveis_dormitorio_1_sp,2577.0
346294,2024-03-31,imoveis_dormitorio_3_sp,3445.0
347647,2024-04-30,imoveis_dormitorio_1_sp,1894.0
348057,2024-04-30,imoveis_dormitorio_3_sp,3533.0


In [21]:
#analisando o filtro buscando seperar a unidade federativa em uma coluna para trabalhar melhor, notamos que existe um problema com imóveis com mais de 4 dormitorios
filtro_imoveis_sp['Info'].str.split('_', expand=True)

,0,1,2,3,4
38,imoveis,valor,avaliacao,sp,None
181,imoveis,area,privativa,sp,None
700,imoveis,dormitorio,3,sp,None
915,imoveis,valor,compra,sp,None
2028,imoveis,dormitorio,2,sp,None
...,...,...,...,...,...
351917,imoveis,dormitorio,2,sp,None
351971,imoveis,dormitorio,3,sp,None
352025,imoveis,dormitorio,4,mais,sp
352077,imoveis,dormitorio,1,sp,None


In [22]:
filtro_imoveis_sp

,Data,Info,Valor
38,2019-10-31,imoveis_valor_avaliacao_sp,215500.00
181,2022-06-30,imoveis_area_privativa_sp,57.95
700,2021-09-30,imoveis_dormitorio_3_sp,6489.00
915,2019-10-31,imoveis_valor_compra_sp,209699.40
2028,2018-07-31,imoveis_dormitorio_2_sp,14570.00
...,...,...,...
351917,2020-01-31,imoveis_dormitorio_2_sp,10307.00
351971,2020-01-31,imoveis_dormitorio_3_sp,3870.00
352025,2020-01-31,imoveis_dormitorio_4_mais_sp,737.00
352077,2020-02-29,imoveis_dormitorio_1_sp,5652.00


In [23]:
filtro_imoveis_sp.loc[:, 'Info'] = filtro_imoveis_sp['Info'].str.strip()

In [24]:
#utilizando Regular Expressions para filtrar os valores problemáticos
filtro_imoveis_4_mais = filtro_imoveis_sp[~filtro_imoveis_sp['Info'].str.contains(r'\wmais\w|\wgarantia\w', regex=True, case= False)]

In [25]:
filtro_imoveis_4_mais = filtro_imoveis_4_mais[filtro_imoveis_4_mais['Info'].str.contains(r'\wdormitorio\w', regex=True, case=False)]

In [26]:
filtro_imoveis_4_mais

,Data,Info,Valor
700,2021-09-30,imoveis_dormitorio_3_sp,6489.0
2028,2018-07-31,imoveis_dormitorio_2_sp,14570.0
2723,2024-02-29,imoveis_dormitorio_2_sp,7370.0
2823,2020-02-29,imoveis_dormitorio_3_sp,3599.0
7374,2018-07-31,imoveis_dormitorio_3_sp,3404.0
...,...,...,...
351863,2020-01-31,imoveis_dormitorio_1_sp,6087.0
351917,2020-01-31,imoveis_dormitorio_2_sp,10307.0
351971,2020-01-31,imoveis_dormitorio_3_sp,3870.0
352077,2020-02-29,imoveis_dormitorio_1_sp,5652.0


In [27]:
filtro_dormitorios = filtro_imoveis_4_mais['Info'].str.split('_', expand=True)

In [28]:
filtro_dormitorios.columns =['categoria', 'operacao', 'quantidade', 'uf' ]

In [29]:
filtro_dormitorios['valor'] = filtro_imoveis_4_mais['Valor']

In [30]:
filtro_dormitorios['data'] = filtro_imoveis_4_mais['Data']

In [31]:
filtro_dormitorios

,categoria,operacao,quantidade,uf,valor,data
700,imoveis,dormitorio,3,sp,6489.0,2021-09-30
2028,imoveis,dormitorio,2,sp,14570.0,2018-07-31
2723,imoveis,dormitorio,2,sp,7370.0,2024-02-29
2823,imoveis,dormitorio,3,sp,3599.0,2020-02-29
7374,imoveis,dormitorio,3,sp,3404.0,2018-07-31
...,...,...,...,...,...,...
351863,imoveis,dormitorio,1,sp,6087.0,2020-01-31
351917,imoveis,dormitorio,2,sp,10307.0,2020-01-31
351971,imoveis,dormitorio,3,sp,3870.0,2020-01-31
352077,imoveis,dormitorio,1,sp,5652.0,2020-02-29


In [32]:
for info in filtro_imoveis_sp['Info']:
    if "mais" in info:
        print(info)

imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormitorio_4_mais_sp
imoveis_dormit

In [33]:
def filtro_unidade_federativa(dataframe,unidade_federativa):
    return dataframe[dataframe['Info'].str.endswith(unidade_federativa)]

In [34]:
filtro_imoveis_rj = filtro_rj(filtro_imoveis)

NameError: name 'filtro_rj' is not defined

,Data,Info,Valor
642,2018-07-31,imoveis_dormitorio_2_rj,2829.0
1337,2024-02-29,imoveis_dormitorio_2_rj,1291.0
1437,2020-02-29,imoveis_dormitorio_3_rj,456.0
2533,2023-02-28,imoveis_garantia_hipoteca_rj,101.0
3998,2019-11-30,imoveis_valor_avaliacao_rj,225000.0
...,...,...,...
351903,2020-01-31,imoveis_dormitorio_2_rj,1999.0
351957,2020-01-31,imoveis_dormitorio_3_rj,453.0
352011,2020-01-31,imoveis_dormitorio_4_mais_rj,119.0
352063,2020-02-29,imoveis_dormitorio_1_rj,906.0


In [ ]:
filtro_imoveis_rr= filtro_unidade_federativa(filtro_imoveis, "mg")

In [ ]:
filtro_imoveis_rr

,Data,Info,Valor
949,2023-02-28,imoveis_garantia_hipoteca_mg,200.0
2414,2019-11-30,imoveis_valor_avaliacao_mg,155000.0
2557,2022-07-31,imoveis_area_privativa_mg,65.3
3076,2021-09-30,imoveis_dormitorio_4_mais_mg,493.0
3291,2019-11-30,imoveis_valor_compra_mg,150596.6
...,...,...,...
351941,2020-01-31,imoveis_dormitorio_3_mg,1195.0
351995,2020-01-31,imoveis_dormitorio_4_mais_mg,243.0
352047,2020-02-29,imoveis_dormitorio_1_mg,793.0
352101,2020-02-29,imoveis_dormitorio_2_mg,2849.0


In [ ]:
filtro_imoveis_rr['Info'].str.split('_', expand=True)

,0,1,2,3,4
949,imoveis,garantia,hipoteca,mg,None
2414,imoveis,valor,avaliacao,mg,None
2557,imoveis,area,privativa,mg,None
3076,imoveis,dormitorio,4,mais,mg
3291,imoveis,valor,compra,mg,None
...,...,...,...,...,...
351941,imoveis,dormitorio,3,mg,None
351995,imoveis,dormitorio,4,mais,mg
352047,imoveis,dormitorio,1,mg,None
352101,imoveis,dormitorio,2,mg,None
